In [27]:
from PIL import Image
from direct.preprocessor import Preprocessor, pose_inv, SceneData
import numpy as np
import open3d as o3d

dir = "experiments/wood"

demo_head_rgb = np.array(Image.open("../{0}/demo_head_rgb.png".format(dir)))
demo_head_depth = np.array(Image.open("../{0}/demo_head_depth.png".format(dir)))
demo_head_mask = np.array(Image.open("../{0}/demo_head_seg.png".format(dir)))

live_head_rgb = np.array(Image.open("../{0}/live_d415_rgb.png".format(dir)))
live_head_depth = np.array(Image.open("../{0}/live_d415_depth.png".format(dir)))
live_head_mask = np.array(Image.open("../{0}/live_d415_seg.png".format(dir)))

intrinsics_d415 = np.load("../handeye/intrinsics_d415.npy")
intrinsics_d405 = np.load("../handeye/intrinsics_d405.npy")
T_WC = np.load("../handeye/T_WC_head.npy")


In [28]:
data = SceneData(
    image_0=demo_head_rgb,
    image_1=live_head_rgb,
    depth_0=demo_head_depth,
    depth_1=live_head_depth,
    seg_0=demo_head_mask,
    seg_1=live_head_mask,
    intrinsics_0=intrinsics_d415,
    intrinsics_1=intrinsics_d415,
    T_WC=np.eye(4) # np.eye(4) = cam frame
)

In [29]:
processor = Preprocessor()
data.update(processor(data))

pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])
o3d.visualization.draw_geometries([pcd0, pcd1])

# Calculate the centroid of each point cloud
pcd0_centre = np.mean(data["pc0"][:, :3], axis=0)  # Calculate mean across columns (axis=0)
pcd1_centre = np.mean(data["pc1"][:, :3], axis=0)  # Calculate mean across columns (axis=0)

print(pcd0_centre)
print(pcd1_centre)
# # Compute the difference between the centroids
diff = pcd0_centre - pcd1_centre
diff

[0.0371766  0.03764415 0.65259916]
[0.0734359  0.04258799 0.6330911 ]


array([-0.0362593 , -0.00494384,  0.01950806], dtype=float32)

In [36]:
import open3d as o3d
import numpy as np
from bayes_opt import BayesianOptimization, UtilityFunction
import time

class Open3dICPPoseEstimator:

    def __init__(self,
                 max_correspondence_distance: float = 0.10,
                 max_iteration: int = 10,
                 timeout=5):
        self.timeout = timeout
        self.max_correspondence_distance = max_correspondence_distance
        self.max_iteration = max_iteration

    def estimate_relative_pose(self,
                               pcd1_o3d: o3d.geometry.PointCloud,
                               pcd2_o3d: o3d.geometry.PointCloud,
                               T_WC: np.ndarray,
                               verbose=True,
                               visualise_pcds=False) -> np.ndarray:

        pcd1_mean = np.asarray(pcd1_o3d.points).mean(axis=0)
        pcd2_mean = np.asarray(pcd2_o3d.points).mean(axis=0)

        def objective_function(roll, pitch, yaw):
            transformation = self.get_transformation_matrix(roll, pitch, yaw, pcd1_mean, pcd2_mean, T_WC)
            reg_p2p = o3d.pipelines.registration.registration_icp(
                pcd1_o3d, pcd2_o3d, self.max_correspondence_distance, transformation,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=self.max_iteration)
            )
            return -reg_p2p.fitness  # Negate fitness to convert maximization to minimization

        pbounds = {'roll': (-np.pi/2, np.pi/2), 'pitch': (-np.pi/2, np.pi/2), 'yaw': (-np.pi/2, np.pi/2)}

        optimizer = BayesianOptimization(
            f=objective_function,
            pbounds=pbounds,
            random_state=42
        )

        utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)

        start = time.time()
        while time.time() - start < self.timeout:
            next_point = optimizer.suggest(utility)
            target = objective_function(**next_point)
            optimizer.register(params=next_point, target=target)
            if verbose:
                print(f'Iteration {len(optimizer.space)} - Roll: {next_point["roll"]:.3f}, Pitch: {next_point["pitch"]:.3f}, '
                      f'Yaw: {next_point["yaw"]:.3f}, Fitness: {-target:.3f}')

        best_params = optimizer.max['params']
        best_transformation = self.get_transformation_matrix(best_params['roll'], best_params['pitch'], best_params['yaw'], pcd1_mean, pcd2_mean, T_WC)

        reg_p2p = o3d.pipelines.registration.registration_icp(
            pcd1_o3d, pcd2_o3d, self.max_correspondence_distance, best_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=self.max_iteration)
        )

        if visualise_pcds:
            self.draw_registration_result(pcd1_o3d, pcd2_o3d, reg_p2p.transformation)

        if reg_p2p.fitness == 0:
            raise Exception('Open3D ICP was unable to register the two point clouds')

        return reg_p2p.transformation

    def get_transformation_matrix(self, roll, pitch, yaw, pcd1_mean, pcd2_mean, T_WC):
        R = self.euler_angles_to_rotation_matrix([roll, pitch, yaw])
        trans_init = np.eye(4)
        trans_init[:3, :3] = R
        trans_init[:3, 3] = T_WC[:3, :3] @ (pcd2_mean - pcd1_mean + 0.01 * np.random.randn(3))
        return trans_init

    @staticmethod
    def euler_angles_to_rotation_matrix(angles):
        roll, pitch, yaw = angles
        R_x = np.array([[1, 0, 0],
                        [0, np.cos(roll), -np.sin(roll)],
                        [0, np.sin(roll), np.cos(roll)]])
        R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])
        R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                        [np.sin(yaw), np.cos(yaw), 0],
                        [0, 0, 1]])
        return R_z @ R_y @ R_x
    @staticmethod
    def draw_registration_result(source, target, transformation):
        source_temp = source.transform(transformation)
        o3d.visualization.draw_geometries([source_temp, target])

pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])

estimator = Open3dICPPoseEstimator()
result = estimator.estimate_relative_pose(pcd0, pcd1, T_WC, verbose=True, visualise_pcds=True)
print("Best transformation:")
print(result)



Iteration 1 - Roll: 1.416, Pitch: -0.394, Yaw: 0.729, Fitness: 0.000
Iteration 2 - Roll: -1.571, Pitch: 1.571, Yaw: -1.571, Fitness: 0.000
Iteration 3 - Roll: 1.548, Pitch: -1.466, Yaw: 1.434, Fitness: 0.000
Iteration 4 - Roll: -1.536, Pitch: 1.497, Yaw: -1.517, Fitness: 0.000
Iteration 5 - Roll: 1.554, Pitch: -1.415, Yaw: 1.515, Fitness: 0.000
Iteration 6 - Roll: -1.481, Pitch: -1.561, Yaw: -1.569, Fitness: 0.000
Iteration 7 - Roll: 1.565, Pitch: 1.463, Yaw: 1.524, Fitness: 0.000
Iteration 8 - Roll: -1.501, Pitch: -1.413, Yaw: -1.555, Fitness: 0.000
Iteration 9 - Roll: 1.524, Pitch: 1.542, Yaw: 1.557, Fitness: 0.000
Iteration 10 - Roll: -1.541, Pitch: -1.407, Yaw: -1.553, Fitness: 0.000
Iteration 11 - Roll: 1.564, Pitch: 1.486, Yaw: 1.425, Fitness: 0.000
Iteration 12 - Roll: -1.565, Pitch: -1.476, Yaw: 1.518, Fitness: 0.000
Iteration 13 - Roll: -1.453, Pitch: 1.556, Yaw: -1.548, Fitness: 0.000
Iteration 14 - Roll: 1.547, Pitch: 1.484, Yaw: -1.502, Fitness: 0.000
Iteration 15 - Roll: -

Exception: Open3D ICP was unable to register the two point clouds

In [37]:
class Open3dICPPoseEstimator:

    def __init__(self,
                 max_correspondence_distance: float = 0.10,
                 max_iteration: int = 10,
                 timeout=15):
        self.timeout = timeout
        self.max_correspondence_distance = max_correspondence_distance
        self.max_iteration = max_iteration

    def estimate_relative_pose(self,
                               pcd1_o3d: o3d.geometry.PointCloud,
                               pcd2_o3d: o3d.geometry.PointCloud,
                               T_WC: np.ndarray,
                               verbose=True,
                               visualise_pcds=False) -> np.ndarray:

        pcd1_mean = np.asarray(pcd1_o3d.points).mean(axis=0)
        pcd2_mean = np.asarray(pcd2_o3d.points).mean(axis=0)

        # Initial SVD alignment
        initial_transform = self.svd_based_initialization(pcd1_o3d, pcd2_o3d)

        def objective_function(roll, pitch, yaw):
            transformation = self.get_transformation_matrix(roll, pitch, yaw, pcd1_mean, pcd2_mean, initial_transform)
            reg_p2p = o3d.pipelines.registration.registration_icp(
                pcd1_o3d, pcd2_o3d, self.max_correspondence_distance, transformation,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=self.max_iteration)
            )
            return -reg_p2p.fitness  # Negate fitness to convert maximization to minimization

        pbounds = {'roll': (-np.pi/4, np.pi/4), 'pitch': (-np.pi/4, np.pi/4), 'yaw': (-np.pi/4, np.pi/4)}

        optimizer = BayesianOptimization(
            f=objective_function,
            pbounds=pbounds,
            random_state=42
        )

        utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)

        start = time.time()
        while time.time() - start < self.timeout:
            next_point = optimizer.suggest(utility)
            target = objective_function(**next_point)
            optimizer.register(params=next_point, target=target)
            if verbose:
                print(f'Iteration {len(optimizer.space)} - Roll: {next_point["roll"]:.3f}, Pitch: {next_point["pitch"]:.3f}, '
                      f'Yaw: {next_point["yaw"]:.3f}, Fitness: {-target:.3f}')

        best_params = optimizer.max['params']
        best_transformation = self.get_transformation_matrix(best_params['roll'], best_params['pitch'], best_params['yaw'], pcd1_mean, pcd2_mean, initial_transform)

        reg_p2p = o3d.pipelines.registration.registration_icp(
            pcd1_o3d, pcd2_o3d, self.max_correspondence_distance, best_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=self.max_iteration)
        )

        if visualise_pcds:
            self.draw_registration_result(pcd1_o3d, pcd2_o3d, reg_p2p.transformation)

        if reg_p2p.fitness == 0:
            raise Exception('Open3D ICP was unable to register the two point clouds')

        return reg_p2p.transformation

    def svd_based_initialization(pcd1_o3d, pcd2_o3d):
        points1 = np.asarray(pcd1_o3d.points)
        points2 = np.asarray(pcd2_o3d.points)

        # Ensure the point clouds have the same number of points by random sampling if necessary
        if len(points1) != len(points2):
            min_points = min(len(points1), len(points2))
            points1 = points1[:min_points]
            points2 = points2[:min_points]

        centroid1 = points1.mean(axis=0)
        centroid2 = points2.mean(axis=0)

        points1_centered = points1 - centroid1
        points2_centered = points2 - centroid2

        H = points1_centered.T @ points2_centered

        U, S, Vt = np.linalg.svd(H)
        R = Vt.T @ U.T

        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.T @ U.T

        t = centroid2.T - R @ centroid1.T

        transformation = np.eye(4)
        transformation[:3, :3] = R
        transformation[:3, 3] = t

        return transformation

    def get_transformation_matrix(self, roll, pitch, yaw, pcd1_mean, pcd2_mean, initial_transform):
        R = self.euler_angles_to_rotation_matrix([roll, pitch, yaw])
        trans_init = np.eye(4)
        trans_init[:3, :3] = R @ initial_transform[:3, :3]
        trans_init[:3, 3] = initial_transform[:3, 3] + 0.01 * np.random.randn(3)
        return trans_init

    @staticmethod
    def euler_angles_to_rotation_matrix(angles):
        roll, pitch, yaw = angles
        R_x = np.array([[1, 0, 0],
                        [0, np.cos(roll), -np.sin(roll)],
                        [0, np.sin(roll), np.cos(roll)]])
        R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])
        R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                        [np.sin(yaw), np.cos(yaw), 0],
                        [0, 0, 1]])
        return R_z @ R_y @ R_x

    @staticmethod
    def draw_registration_result(source, target, transformation):
        source_temp = source.transform(transformation)
        o3d.visualization.draw_geometries([source_temp, target])

# Example usage
# data["pc0"] and data["pc1"] are your point cloud data arrays.
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])

estimator = Open3dICPPoseEstimator()
result = estimator.estimate_relative_pose(pcd0, pcd1, T_WC, verbose=True, visualise_pcds=True)
print("Best transformation:")
print(result)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 22816 is different from 21767)

In [48]:
from scipy.spatial.transform import Rotation as R

def create_homogeneous_matrix(xyz, quaternion):
    # Convert the quaternion to a rotation matrix
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    # Create a homogeneous transformation matrix
    T = np.eye(4)  # Start with an identity matrix
    T[:3, :3] = rotation_matrix  # Insert the rotation matrix
    T[:3, 3] = xyz  # Insert the translation vector

    return T

simple_T = create_homogeneous_matrix(diff, [0, 0, 0, 1])
T_WC = np.load("../handeye/T_WC_head.npy")
T_delta_world = T_WC @ simple_T @ pose_inv(T_WC)
T_delta_world

array([[ 1.00000000e+00,  5.76610702e-18,  4.24064341e-18,
        -5.48792420e-02],
       [ 5.76610702e-18,  1.00000000e+00,  3.26733594e-17,
         1.82634440e-01],
       [ 4.24064341e-18,  3.26733594e-17,  1.00000000e+00,
         6.19532222e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [26]:
import open3d as o3d
import numpy as np
import copy
import cv2
import time

start = time.time()
# Assuming data["pc0"] and data["pc1"] are your point cloud data
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])

# # Estimate normals for each point cloud
# pcd0.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))
# pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))

# Function to draw registration results
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

# Compute FPFH features
voxel_size = 0.05  # Set voxel size for downsampling (adjust based on your data)
source_down = pcd0.voxel_down_sample(voxel_size)
target_down = pcd1.voxel_down_sample(voxel_size)

source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

# Global registration using RANSAC
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, mutual_filter=False,
    max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
    ransac_n=4,
    checkers=[
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), 
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
)

# Use the result of global registration as the initial transformation for ICP
trans_init = result.transformation
# Apply ICP
threshold = 0.01  # Set a threshold for ICP, this depends on your data
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd0, pcd1, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

# Get the transformation matrix
T_delta_cam = reg_p2p.transformation

print(time.time() - start)
# Draw the result
draw_registration_result(pcd0, pcd1, T_delta_cam)

print(T_delta_cam)

1.920013189315796
[[ 0.85319111 -0.51953292 -0.04637329  0.1287207 ]
 [ 0.51371596  0.85236793 -0.09779997 -0.2560663 ]
 [ 0.09033741  0.05961937  0.99412508 -0.04125192]
 [ 0.          0.          0.          1.        ]]


In [51]:
start = time.time()
# Assuming data["pc0"] and data["pc1"] are your point cloud data
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])

# Estimate normals for each point cloud
# pcd0.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))
# pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))

# Function to draw registration results
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

# Compute FPFH features
voxel_size = 0.005  # Set voxel size for downsampling (adjust based on your data)
source_down = pcd0.voxel_down_sample(voxel_size)
target_down = pcd1.voxel_down_sample(voxel_size)

source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)

# Use the result of global registration as the initial transformation for ICP
trans_init = result_fast.transformation
# Apply ICP
threshold = 0.01  # Set a threshold for ICP, this depends on your data
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd0, pcd1, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

# Get the transformation matrix
T_delta_cam = reg_p2p.transformation


print(time.time() - start)
# Draw the result
draw_registration_result(pcd0, pcd1, T_delta_cam)

print(T_delta_cam)

:: Apply fast global registration with distance threshold 0.003
0.17958688735961914
[[ 0.9623048   0.12707494 -0.24046086 -0.02170186]
 [-0.12790808  0.99171092  0.01220592  0.0214484 ]
 [ 0.24001873  0.01901107  0.97058209 -0.0313623 ]
 [ 0.          0.          0.          1.        ]]


In [62]:
T_WC = np.load("../handeye/T_WC_head.npy")
T_delta_world = T_WC @ T_delta_cam @ pose_inv(T_WC)
T_delta_world

array([[ 0.96091092, -0.27669804, -0.00940215,  0.06690749],
       [ 0.27684587,  0.96062984,  0.02338089, -0.31354479],
       [ 0.00256254, -0.0250699 ,  0.99968242, -0.0022233 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [61]:
r = R.from_matrix(T_delta_world[:3, :3]).as_euler("xyz")
yaw_only_delta_rotation = R.from_euler("xyz", [0, 0, r[-1]]).as_matrix()
yaw_only_delta_rotation

array([[ 0.96091408, -0.27684678,  0.        ],
       [ 0.27684678,  0.96091408,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [5]:
from scipy.spatial.transform import Rotation as R


def translation_from_matrix(matrix):
    """Extracts the translation vector from a 4x4 homogeneous transformation matrix."""
    return matrix[:3, 3]

def euler_from_matrix(matrix):
    """Extracts the quaternion from a 4x4 homogeneous transformation matrix."""
    rotation_matrix = matrix[:3, :3].copy()
    rotation = R.from_matrix(rotation_matrix)
    return rotation.as_euler(seq="XYZ", degrees=True)

trans = translation_from_matrix(T_delta_world)
rotation = euler_from_matrix(T_delta_world)

print(trans, rotation)

NameError: name 'T_delta_world' is not defined

In [17]:
def create_homogeneous_matrix(xyz, quaternion):
    # Convert the quaternion to a rotation matrix
    rotation_matrix = R.from_quat(quaternion).as_matrix()
    # Create a homogeneous transformation matrix
    T = np.eye(4)  # Start with an identity matrix
    T[:3, :3] = rotation_matrix  # Insert the rotation matrix
    T[:3, 3] = xyz  # Insert the translation vector

    return T

demo = [
    0.5041812568964179,
    0.09185436015924689,
    0.48143709451847916,
    -0.9983786626178943,
    0.013449729176136651,
    -0.054892707858185244,
    0.006778011389003352
]

T_eef = create_homogeneous_matrix(demo[:3], demo[3:])
T_eef

array([[ 0.99361179, -0.02611172,  0.10978974,  0.50418126],
       [-0.02759997, -0.99954633,  0.01205746,  0.09185436],
       [ 0.10942509, -0.01501063, -0.9938817 ,  0.48143709],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [35]:
T_delta_world @ T_eef

array([[ 0.80887007,  0.58083391,  0.09143946,  0.62268341],
       [ 0.57427787, -0.81378487,  0.08921382, -0.0129589 ],
       [ 0.12623046, -0.01965073, -0.99180629,  0.48391332],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [36]:
new_T @ T_eef

array([[ 0.74345614, -0.66244621,  0.09185851,  0.62268341],
       [-0.65977202, -0.74895908, -0.06132849, -0.0129589 ],
       [ 0.10942509, -0.01501063, -0.9938817 ,  0.48391332],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [56]:
T_bias = create_homogeneous_matrix([trans[0], trans[1], 0], R.from_euler('xyz', [0, 0, rotation[2]]).as_quat())
T_bias

array([[ 0.76608952,  0.64273389,  0.        ,  0.28067213],
       [-0.64273389,  0.76608952, -0.        , -0.38310653],
       [-0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [62]:
R.from_matrix(T_eef[:3, :3]).as_euler('xyz')

array([-3.12649077, -0.10964465, -0.02777028])

In [61]:
res = T_bias @ T_eef
R.from_matrix(res[:3, :3]).as_euler('xyz')


array([-3.12649077, -0.10964465, -0.72583185])

In [52]:
R.from_euler('xyz', [0, 0, rotation[2]]).as_matrix() @ T_eef[:3, :3]

array([[ 0.74345614, -0.66244621,  0.09185851],
       [-0.65977202, -0.74895908, -0.06132849],
       [ 0.10942509, -0.01501063, -0.9938817 ]])

In [37]:
def pose_inv(pose):
    """Inverse a 4x4 homogeneous transformation matrix."""
    R = pose[:3, :3]
    T = np.eye(4)
    T[:3, :3] = R.T
    T[:3, 3] = - R.T @ np.ascontiguousarray(pose[:3, 3])
    return T

In [44]:
import numpy as np
r_project = np.linalg.inv(R.from_euler('xyz', [rotation[0], rotation[1], rotation[2]]).as_matrix()) @ R.from_euler('xyz', [0, 0, rotation[2]]).as_matrix()
r_project

array([[ 8.04888450e-01, -6.66575509e-17,  5.93426139e-01],
       [-4.17340537e-01,  7.10919961e-01,  5.66056256e-01],
       [-4.21878488e-01, -7.03272926e-01,  5.72211266e-01]])

In [20]:
r = R.from_euler('xyz', [0, 0, rotation[-1]])

# Get the rotation matrix
adjusted_rotation_matrix = r.as_matrix()

adjusted_rotation_matrix

array([[ 0.76608952,  0.64273389,  0.        ],
       [-0.64273389,  0.76608952, -0.        ],
       [-0.        ,  0.        ,  1.        ]])

In [31]:
adjusted_translation = T_delta_world[:3, :3] @ T_eef[:3, 3] - adjusted_rotation_matrix @ T_eef[:3, 3] + T_delta_world[:3, 3]
adjusted_translation

array([0.17739752, 0.24072682, 0.00247622])

In [32]:
new_T =  create_homogeneous_matrix(adjusted_translation, r.as_quat())
new_T

array([[ 0.76608952,  0.64273389,  0.        ,  0.17739752],
       [-0.64273389,  0.76608952, -0.        ,  0.24072682],
       [-0.        ,  0.        ,  1.        ,  0.00247622],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [27]:
T_delta_world

array([[ 0.79857538, -0.60179267, -0.011088  ,  0.28067213],
       [ 0.60165335,  0.79864132, -0.01361215, -0.38310653],
       [ 0.01704703,  0.00419919,  0.99984587, -0.00643008],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [133]:
PointCloud = np.ndarray

def rotate_pointcloud(pcd: PointCloud, angle_z: float):
    print("predicted rotation", np.rad2deg(angle_z))
    R = np.eye(3)
    cosine = np.cos(angle_z)
    sine = np.sin(angle_z)
    R[0, 0] = cosine
    R[1, 1] = cosine
    R[0, 1] = -sine
    R[1, 0] = sine

    pcd[:3, :] = R @ pcd[:3, :]
    return R, pcd

def find_translation(pcd0: PointCloud, pcd1: PointCloud) -> np.ndarray:
    pcd0_centre = np.mean(pcd0[:3, :], axis=1)
    pcd1_centre = np.mean(pcd1[:3, :], axis=1)
    return pcd1_centre - pcd0_centre
    
R_mtx, rotated_pcd0 = rotate_pointcloud(data["pc0"], rotation[-1])
translation = find_translation(rotated_pcd0, data["pc1"])

print(translation)
T_delta_base = np.eye(4)
T_delta_base[:3, :3] = R_mtx
T_delta_base[:3, 3] = translation

T_delta_cam = pose_inv(data["T_WC"]) @ T_delta_base @ data["T_WC"]

print(T_delta_base)

predicted rotation -1854.694161068739
[-0.09844735 -0.39142355 -0.21084177]
[[ 0.57794079  0.8160787   0.         -0.09844735]
 [-0.8160787   0.57794079  0.         -0.39142355]
 [ 0.          0.          1.         -0.21084177]
 [ 0.          0.          0.          1.        ]]


In [63]:
import open3d as o3d
import numpy as np

width = 848  # Replace with your camera image width
height = 480 # Replace with your camera image height
fx = 431.56503296
fy = 431.18637085
cx = 418.71490479
cy = 235.15617371 
intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

demo_mask = np.array(Image.open("../data/lego/demo_wrist_mask.png"))
demo_rgb = np.array(Image.open("../data/lego/demo_wrist_rgb.png")) 
demo_depth = np.array(Image.open("../data/lego/demo_wrist_depth.png")).astype(np.uint16)

color = o3d.geometry.Image(demo_rgb)
depth = o3d.geometry.Image(demo_depth)

rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth, depth_scale=1000.0, convert_rgb_to_intensity=False)

rgbd_image
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsics) 

# Visualization
o3d.visualization.draw_geometries([pcd]) 